In [1]:
import os
import pandas as pd
import geopandas as gpd
# from osgeo import gdal
# import contextily as ctx
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import DescrStatsW
import sumolib
import scienceplots
plt.style.use(['science', 'no-latex'])
from aux_functions import load_pickle

In [2]:
scr = 'Zurich'

In [3]:
standard_input = load_pickle(r'data\{}\input\standard_input.pkl'.format(scr))
detectors_standard_input = load_pickle(r'data\{}\input\detectors_standard_input.pkl'.format(scr))

In [4]:
# define directory
result_dir = r'results\{}\scr_evaluation'.format(scr)
if not os.path.isdir(result_dir):
    os.makedirs(result_dir)

# Network Features

In [5]:
gdf = gpd.GeoDataFrame(standard_input[['edge_id', 'lane_id', 'geometry']].drop_duplicates(), geometry='geometry', crs='EPSG:4326').to_crs(crs='EPSG:25832')

In [6]:
network_bounds = gdf.total_bounds
print('Network extent:', round((network_bounds[2] - network_bounds[0])/1000), 'km', 'X',round((network_bounds[3] - network_bounds[1])/1000), 'km')

Network extent: 13 km X 14 km


In [7]:
net_lane_length = gdf.length.sum()
net_edge_length = gdf[['edge_id', 'geometry']].drop_duplicates().length.sum()

In [8]:
print('Network lane length: {:.0f} ln-km'.format(net_lane_length/1000))
print('Network edge length: {:.0f} km'.format(net_edge_length/1000))

Network lane length: 979 ln-km
Network edge length: 769 km


In [9]:
print('No. of lanes:', gdf.lane_id.nunique())
print('No. of edges:', gdf.edge_id.nunique())

No. of lanes: 9320
No. of edges: 7237


In [10]:
edges = gdf.edge_id.unique()
nodes = load_pickle(r'data\{}\network\nodes.pkl'.format(scr)).explode('connected_edges')
print('No. of nodes:', nodes.loc[nodes.connected_edges.isin(edges)].node_id.nunique())

No. of nodes: 3530


# Network

In [11]:
network_gdf = gpd.GeoDataFrame(standard_input[['edge_id', 'geometry']].drop_duplicates(), geometry='geometry', crs= 'EPSG:4326').to_crs('EPSG:25832')
detectors_gdf = gpd.GeoDataFrame(detectors_standard_input[['detector_id', 'geometry']].drop_duplicates(), geometry='geometry', crs= 'EPSG:4326').to_crs('EPSG:25832')

# plot
fig, ax = plt.subplots(figsize=(2, 2), dpi=1200)

network_gdf.plot(ax=ax, linewidth=0.15)
detectors_gdf.plot(ax=ax, color='red', edgecolor='none', markersize=0.2)

# add a base map
# ctx.add_basemap(ax, crs=network_gdf.crs, source='CartoDB.Positron', attribution_size=1, zoom=16)

# format
ax.set_axis_off()
ax.grid(False)
for spine in ax.spines.values():
    spine.set_visible(False)

# save plot
plt.savefig(os.path.join(result_dir, 'network.png'), bbox_inches='tight')
plt.close()

# MFD 

In [12]:
stats = []
for timestep, temp in standard_input.groupby('timestep'):
    speed = DescrStatsW(temp.speed, weights=temp.sampled_seconds)
    density = DescrStatsW(temp.density, weights=temp.length)
    flow = DescrStatsW(temp.flow, weights=temp.length)

    stats.append(
        (timestep, 
         speed.mean, speed.std, speed.var,
         density.mean, density.std, density.var,
         flow.mean, flow.var, flow.std)
    )

mfd = pd.DataFrame(stats, columns=['timestep', 
                                    'speed_mean', 'speed_std', 'speed_var',
                                    'density_mean', 'density_std', 'density_var',
                                    'flow_mean', 'flow_var', 'flow_std']
                    )

In [13]:
fig, ax = plt.subplots(figsize=(4, 4), dpi=600)
sc = ax.scatter(mfd.density_mean, mfd.flow_mean, c =mfd.timestep, s=4, cmap='viridis', edgecolor='none')

# Add color bar
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('Timestep [sec]', fontsize=4)   
cbar.ax.tick_params(which='major', labelsize=4, pad=1, length=1.2, width=0.3)
cbar.ax.tick_params(which='minor', length=0.8, width=0.2)
cbar.outline.set_visible(False)

# format
ax.set_xlabel('Density [veh/km-ln]', fontsize=6, labelpad=2)
ax.set_ylabel('Flow [veh/hr-ln]', fontsize=6, labelpad=2)

ax.set_xlim(0)
ax.set_ylim(0)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.minorticks_off()
ax.tick_params(axis='both', which='both', labelsize=6, pad=2, top=False, right=False)

plt.savefig(os.path.join(result_dir, 'mfd.png'), bbox_inches='tight')
plt.close('all')

# Density Over Time

In [14]:
fig, ax = plt.subplots(figsize=(10, 5), dpi=600)

# plot
ax.plot(mfd.timestep, mfd.density_mean, linewidth=1, alpha=0.2, label='Density [veh/km-ln]', color='grey')
ax.scatter(mfd.timestep, mfd.density_mean, c =mfd.timestep, s=4, cmap='viridis', edgecolor='none')

ax.plot(mfd.timestep, mfd.density_std, alpha=0.2, color='grey', linestyle='--', label='Density SD [veh/km-ln]')

# Add color bar
cbar = plt.colorbar(sc, ax=ax)
cbar.set_label('Timestep [sec]', fontsize=8)   
cbar.ax.tick_params(which='major', labelsize=8, pad=1, length=1.2, width=0.3)
cbar.ax.tick_params(which='minor', length=0.8, width=0.2)
cbar.outline.set_visible(False)

# format
ax.set_xlabel('Time [s]', fontsize=8, labelpad=4)
ax.set_ylabel('Density [veh/km-ln]', fontsize=8, labelpad=4)

ax.set_xlim(0, mfd.timestep.max())
ax.set_ylim(0)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.minorticks_off()
ax.tick_params(axis='both', which='both', labelsize=8, pad=4, top=False, right=False)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)

# add a legend
ax.legend(fontsize=8, loc='upper left')

# save the plot
plt.savefig(os.path.join(result_dir, 'density.png'), bbox_inches='tight')
plt.close('all')


# No. of running vehicles over time

In [15]:
file_path = r'scenarios\{}\output\summary_output.xml'.format(scr)
times = []
no_vehs = []
for time, no_veh in sumolib.xml.parse_fast(file_path, 'step', ['time', 'running']):
    times.append(sumolib.miscutils.parseTime(time))
    no_vehs.append(float(no_veh))

In [16]:
fig, ax = plt.subplots(figsize=(10, 5), dpi=600)
ax.plot(times, no_vehs, color='blue', label='Running Vehicles [#]')

ax.set_xlabel('Time [sec]', fontsize=8, labelpad=4)
ax.set_ylabel('Running Vehicles [#]', fontsize=8, labelpad=4)

ax.set_xlim(0, max(times))
ax.set_ylim(0, max(no_vehs) + 200)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.minorticks_off()
ax.tick_params(axis='both', which='both', labelsize=8, pad=4, top=False, right=False)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.legend(fontsize=8)

plot_path = os.path.join(result_dir, 'running_vehicles.png').replace(os.sep, '/')
plt.savefig(plot_path, bbox_inches='tight')
plt.close()

# Teleporting over time

In [17]:
file_path = r'scenarios\{}\output\summary_output.xml'.format(scr)
times = []
no_vehs = []
for time, no_veh in sumolib.xml.parse_fast(file_path, 'step', ['time', 'teleports']):
    times.append(sumolib.miscutils.parseTime(time))
    no_vehs.append(float(no_veh))

In [18]:
fig, ax = plt.subplots(figsize=(10, 5), dpi=600)
ax.plot(times, no_vehs, color='blue', label='Teleported Vehicles [#]')

ax.set_xlabel('Time [sec]', fontsize=8, labelpad=4)
ax.set_ylabel('Teleported Vehicles [#]', fontsize=8, labelpad=4)

ax.set_xlim(0, max(times))
ax.set_ylim(0, max(no_vehs) + 200)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.minorticks_off()
ax.tick_params(axis='both', which='both', labelsize=8, pad=4, top=False, right=False)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.legend(fontsize=8)

plot_path = os.path.join(result_dir, 'teleported_vehicles.png').replace(os.sep, '/')
plt.savefig(plot_path, bbox_inches='tight')
plt.close()

# Density Over Space (Heatmap)

In [19]:
from matplotlib.colors import Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable

# create GeoDataFrame
gdf = gpd.GeoDataFrame(standard_input[standard_input.timestep==3300], geometry='geometry', crs='EPSG:4326').to_crs('EPSG:25832')

# plot
fig, ax = plt.subplots(figsize=(2, 2), dpi=600)
cax = gdf.plot(
    ax=ax,
    column='density',
    linewidth=0.15,
    cmap='Reds',
    legend=False  # disable the default legend
)

# add colorbar manually
norm = Normalize(vmin=gdf['density'].min(), vmax=gdf['density'].max())
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0)
sm = plt.cm.ScalarMappable(cmap='Reds', norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, cax=cax)
cbar.set_label("Density [veh/km-ln]", fontsize=4)   
cbar.ax.tick_params(which='major', labelsize=4, pad=1, length=1, width=0.2)
cbar.ax.tick_params(which='minor', length=0.5, width=0.1)

# add basemap
# ctx.add_basemap(ax, crs=gdf.crs, source='CartoDB.Positron', attribution_size=1, zoom=16)

# remove the colorbar frame
cbar.outline.set_visible(False)

# format
ax.minorticks_off()
ax.set_axis_off()
ax.grid(False)
for spine in ax.spines.values():
    spine.set_visible(False)

# save the plot
plt.savefig(os.path.join(result_dir, 'density_heatmap'), bbox_inches='tight')
plt.close()

In [20]:
mfd.to_csv(os.path.join(result_dir, 'mfd.csv'), index=False)
standard_input.to_csv(os.path.join(result_dir, 'standard_input.csv'), index=False)